# Семинар 6: Policy gradient


## Майнор ВШЭ, 28.02.2019

Опрос на сегодня - https://goo.gl/forms/RrNM85N4xLTkb7yg1.

## Алгоритм Reinforce

В некоторых задачах для нахождения удовлетворяющей стратегии необязательно изучать структуру всей среды. Например, в задаче поднятия кубика робототехнической рукой вместо точной аппроксимации $Q(s,a)$ достаточно знать, что выгоднее двигаться вправо, если кубик справа, и влево в ином случае. Алгоритм Reinforce (Monte Carlo policy gradient) - это алгоритм поиска стратегий, в котором параметры, задающие стохастическую стратегию, изменяются в соответствии с градиентом математического ожидания награды: 

$J(\theta)=\mathbb E_{\tau\sim p_{\theta}(\tau)}(\sum_t\gamma^tr(s_t,a_t))$

$\theta\leftarrow\theta+\alpha\nabla_{\theta}J(\theta)$

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import gym
env = #Создайте среду Acrobot-v1
observation_shape = env.observation_space.shape
n_actions = env.action_space.n
gamma = 0.95

print("Observation Space", env.observation_space)
print("Action Space", env.action_space)

In [ ]:
#Задайте переменные, которые будут подаваться на вход нейронной сети
#Состояния
observations =
#Совершенные действия
actions =
# "G = r + gamma*r' + gamma^2*r'' + ..."
discounted_episode_rewards =
all_inputs = [observations, actions, discounted_episode_rewards]

### Задание 1. Определяем архитектуру сети

Стратегия задается весами нейронной сети и является стохастической, т.е. в состоянии $s$ она представляет себой некоторое распределение $\pi_\theta(s)$, поэтому на вход сети будет родаваться состояние $s$, а на выходе будут вероятности действий.

In [ ]:
sess = tf.InteractiveSession()
#Задайте внутренние и выходной слои нейронной сети. 
nn1 = 
nn2 = 
#Выход последнего слоя преобразуется в стохастическую стратегию, поэтому количество нейронов должно быть соответствующим
nn3 =  

probs_out = tf.nn.softmax(nn3) 

In [ ]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    
    return discounted_episode_rewards

### Задание 2. Функция потерь

Теперь определите функцию потерь (Crossentropy loss)

Градиент стратегии выглядит следующим образом $\nabla_{\theta}J_{\theta}\approx\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^T\nabla_{\theta}\log\pi_{\theta}(a_{i,t}|s_{i,t})R $. Чтобы автоматически вычислить градиент, необходимо задать граф, который имеет градиент такого же вида. Для этого будем использовать "псевдо" функцию потерь: $\tilde{J}_{\theta}=\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^T\log\pi_{\theta}(a_{i,t}|s_{i,t})R $.

In [ ]:
neg_log_prob = #примените кросс энтропию к последнему слою сети tf.nn.softmax_cross_entropy_with_logits_v2
loss = #умножьте на дисконтированную награду и возьмите среднее, чтобы получить искомую функцию потерь
    
optimizer = #задайте метод оптимизации, который хотите использовать
train_op = optimizer.minimize(loss, global_step=tf.contrib.framework.get_global_step())
sess.run(tf.global_variables_initializer())

Градиент имеет большую дисперсию. Для ее уменьшения можно использовать следующую функцию потерь: $\tilde{J}_{\theta}=\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^Tlog\pi_{\theta}(a_{i,t}|s_{i,t})(R-b) $.

$b$ - может быть константой (это не сильно улучшит работу алгоритма), а может быть функцией от $s$, например $V(s)$. $V(s)$ может быть аппроксимированна другой нейронной сетью. Настройка может проходить по методу наименьших квадратов: необходимо задать функцию ошибок и можно добавить ее к loss'у с некоторым коэффициентом или минимизировать отдельно. Пример: https://github.com/yrlu/reinforcement_learning/blob/master/policy_gradient/reinforce_w_baseline.py

In [ ]:
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
episode_states, episode_actions, episode_rewards = [],[],[]
max_episodes = 10000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for episode in range(max_episodes):
        
        episode_rewards_sum = 0
        state = env.reset()
        env.render()
           
        while True:
            
            probs = #Получите распределение вероятностей действий в соответствии со стохастической стратегией агента
            
            action = #Выберете действие (по распределению) 

            new_state, reward, done, info = 

            episode_states.append(state)
            action_ = np.zeros(n_actions)
            action_[action] = 1
            
            episode_actions.append(action_)
            
            episode_rewards.append(reward)
            if done:
                episode_rewards_sum = np.sum(episode_rewards)
                
                allRewards.append(episode_rewards_sum)
                
                total_rewards = np.sum(allRewards)
                
                
                mean_reward = np.divide(total_rewards, episode+1)
                
                
                maximumRewardRecorded = np.amax(allRewards)
                
                print("==========================================")
                print("Episode: ", episode)
                print("Reward: ", episode_rewards_sum)
                print("Mean Reward", mean_reward)
                print("Max reward so far: ", maximumRewardRecorded)
                
                discounted_rewards = #Посчитайте дисконтированную награду
                                
                #Feedforward, gradient and backpropagation
                loss_, _ = #Посчитайте значение функции потерь и настройте веса сети при помощи оптимизатора
                episode_states, episode_actions, episode_rewards = [],[],[]
                
                break
            
            state = new_state

### Задание 3. Проводим эксперименты.

Постройте график получаемого суммарного вознаграждения от номера эпизода.

Сделайте тоже самое для задачи CartPole-v0. 

Усложните архитектуру сети. Улучшает ли это производительность?

### Дополнительные материалы

Пример алгоритма DDPG (https://arxiv.org/pdf/1509.02971.pdf) - алгоритм использующий градиент стратегий и позволяющий работать с непрерывным множеством действий:

https://pemami4911.github.io/blog/2016/08/21/ddpg-rl.html